In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from string import ascii_lowercase
from itertools import combinations

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('./data/train.csv').drop('index',axis=1)
# train.drop('introelapse',axis=1,inplace=True)
# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['familysize']>100].index.to_list())
# train = train.drop(train[train['introelapse']>3000].index.to_list())
# train = train.drop(train[train['testelapse']>10000].index.to_list())
# train = train.drop(train[train['surveyelapse']>10000].index.to_list()) # 나에 대한 추가 질문

test = pd.read_csv('./data/test.csv').drop('index',axis=1)
# test.drop('introelapse',axis=1,inplace=True)
test.loc[test['familysize']>100,'familysize']= train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()
# test.loc[test['introelapse']>3000,'introelapse']= train['introelapse'].mean()
# test.loc[test['testelapse']>10000,'testelapse']= train['testelapse'].mean()
# test.loc[test['surveyelapse']>10000,'surveyelapse']= train['surveyelapse'].mean()

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>1000 else (2 if x>500 else (3 if x>200 else (4 if x>100 else 5))),value))
temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])


train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] - train['Q3'] - train['Q6'] - train['Q7'] - train['Q10'] + train['Q12'] + train['Q15'] - train['Q16']
train['V'] = -train['Q4'] + train['Q5'] + train['Q8'] - train['Q11'] + train['Q13'] - train['Q14'] - train['Q17'] + train['Q18'] + train['Q20']
train['M'] = -train['Q9'] + train['Q19']
train['introelapse'] = np.log1p(train['introelapse'])
train['testelapse'] = np.log1p(train['testelapse'])
train['surveyelapse'] = np.log1p(train['surveyelapse'])
train.drop('hand',axis=1,inplace=True)
train.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)

# train.drop(['country'],axis=1,inplace=True)

## 6,7,8,9,11,12 의미 없어 보임
# 6,8,10 실존하지 않는 단어 -> 얘네는 의미있게 작용하지 않을까?
# -> 7,8,11 제거

# train.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# train.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

# train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# train.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
# train.drop(['country','hand','introelapse','testelapse','surveyelapse'],axis=1,inplace=True)
# train.drop(['introelapse','testelapse','surveyelapse'],axis=1,inplace=True)
train_fill_na = train.fillna(train.mean())


value = test['country'].value_counts().values
# rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
rank = list(map(lambda x : 1 if x>1000 else (2 if x>500 else (3 if x>200 else (4 if x>100 else 5))),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] - test['Q3'] - test['Q6'] - test['Q7'] - test['Q10'] + test['Q12'] + test['Q15'] - test['Q16']
test['V'] = -test['Q4'] + test['Q5'] + test['Q8'] - test['Q11'] + test['Q13'] - test['Q14'] - test['Q17'] + test['Q18'] + test['Q20']
test['M'] = -test['Q9'] + test['Q19']

test['introelapse'] = np.log1p(test['introelapse'])
test['testelapse'] = np.log1p(test['testelapse'])
test['surveyelapse'] = np.log1p(test['surveyelapse'])
test.drop('hand',axis=1,inplace=True)
test.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# test.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# test.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# test.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
# test.drop(['country','hand','introelapse','testelapse','surveyelapse'],axis=1,inplace=True)
test.drop(['country'],axis=1,inplace=True)


test_fill_na = test.fillna(test.mean())

In [4]:
from pycaret.classification import *

clf = setup(data = train, target = 'nerdiness',remove_outliers=True,fix_imbalance=True)

: 

: 

In [1]:
best_3 = compare_models(sort = 'AUC', n_select = 2)

NameError: name 'compare_models' is not defined

In [ ]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
predictions

In [ ]:
index = test_index

submission = pd.DataFrame({
    "index" : index,
    "nerdiness" : predictions['Score']
})
submission.to_csv('./data/model_auto_ml.csv', index=False)

In [ ]:
y_train = train_fill_na['nerdiness']
x_train = train_fill_na.drop('nerdiness',axis=1)

In [ ]:
test = test_fill_na

In [ ]:
k_fold = KFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
from catboost import CatBoostClassifier

params = {'bagging_temperature': 0.375906, 'depth': 9.0, 'l2_leaf_reg': 68.8, 'learning_rate': 0.011, 'subsample': 0.76046}

clf1 = CatBoostClassifier(**params, iterations=5000, eval_metric='AUC',allow_writing_files=False,od_type='Iter',random_state=777)
clf2 = LGBMClassifier()
clf3 = GradientBoostingClassifier()
soft_vote  = VotingClassifier([('r1',clf1), ('r2', clf2), ('r3',clf3)], voting='soft')
soft_vote.fit(x_train, y_train)

In [ ]:
model = soft_vote
pred_y = model.predict_proba(test_fill_na)
pred_y = pred_y[:,1]

index = test_index

submission = pd.DataFrame({
    "index" : index,
    "voted" : pred_y
})
submission.to_csv('./data/model1.csv', index=False)

# model2

In [126]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv').drop('index',axis=1)
# train.drop('introelapse',axis=1,inplace=True)
# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['familysize']>100].index.to_list())
# train = train.drop(train[train['introelapse']>3000].index.to_list())
# train = train.drop(train[train['testelapse']>10000].index.to_list())
# train = train.drop(train[train['surveyelapse']>10000].index.to_list()) # 나에 대한 추가 질문

test = pd.read_csv('./data/test.csv').drop('index',axis=1)
# test.drop('introelapse',axis=1,inplace=True)
test.loc[test['familysize']>100,'familysize']= train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()
# test.loc[test['introelapse']>3000,'introelapse']= train['introelapse'].mean()
# test.loc[test['testelapse']>10000,'testelapse']= train['testelapse'].mean()
# test.loc[test['surveyelapse']>10000,'surveyelapse']= train['surveyelapse'].mean()

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>1000 else (2 if x>500 else (3 if x>200 else (4 if x>100 else 5))),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] - train['Q3'] - train['Q6'] - train['Q7'] - train['Q10'] + train['Q12'] + train['Q15'] - train['Q16']
train['V'] = -train['Q4'] + train['Q5'] + train['Q8'] - train['Q11'] + train['Q13'] - train['Q14'] - train['Q17'] + train['Q18'] + train['Q20']
train['M'] = -train['Q9'] + train['Q19']
# train['introelapse'] = np.log1p(train['introelapse'])
# train['testelapse'] = np.log1p(train['testelapse'])
# train['surveyelapse'] = np.log1p(train['surveyelapse'])
# train.drop('hand',axis=1,inplace=True)
# train.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)

## 6,7,8,9,11,12 의미 없어 보임
# 6,8,10 실존하지 않는 단어 -> 얘네는 의미있게 작용하지 않을까?
# -> 7,8,11 제거

# train.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# train.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# train.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
# train.drop(['country','hand','introelapse','testelapse','surveyelapse'],axis=1,inplace=True)
train_fill_na = train.fillna(train.mean())


value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>1000 else (2 if x>500 else (3 if x>200 else (4 if x>100 else 5))),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] - test['Q3'] - test['Q6'] - test['Q7'] - test['Q10'] + test['Q12'] + test['Q15'] - test['Q16']
test['V'] = -test['Q4'] + test['Q5'] + test['Q8'] - test['Q11'] + test['Q13'] - test['Q14'] - test['Q17'] + test['Q18'] + test['Q20']
test['M'] = -test['Q9'] + test['Q19']

# test['introelapse'] = np.log1p(test['introelapse'])
# test['testelapse'] = np.log1p(test['testelapse'])
# test['surveyelapse'] = np.log1p(test['surveyelapse'])
# test.drop('hand',axis=1,inplace=True)
# test.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# test.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# test.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# test.drop(['country','hand','introelapse','testelapse','surveyelapse'],axis=1,inplace=True)
# test.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
test_fill_na = test.fillna(test.mean())

In [127]:
temp_ls = train_fill_na.columns.to_list()
temp_ls.remove('mach_score')
temp_ls.remove('nature_score')
temp_ls.remove('nerdiness')
temp_ls.remove('country')
temp_ls.remove('introelapse')
temp_ls.remove('testelapse')
temp_ls.remove('surveyelapse')
temp_ls.remove('hand')

In [128]:
for i in temp_ls:
    train_fill_na[i] = train_fill_na[i].astype(int)
    test_fill_na[i] = test_fill_na[i].astype(int)

In [129]:
y_train = train_fill_na['nerdiness']
x_train = train_fill_na.drop('nerdiness',axis=1)

In [130]:
test = test_fill_na

In [131]:
import pandas as pd
import numpy as np

from string import ascii_lowercase
from itertools import combinations

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

In [132]:
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
import eli5
from eli5.sklearn import PermutationImportance

import matplotlib.pyplot as plt

import warnings
import gc
warnings.filterwarnings("ignore")

from sklearn.ensemble import ExtraTreesClassifier

In [133]:
def lgbm_rfe_777(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        # model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        model = RandomForestClassifier(n_estimators=1000,n_jobs=32,random_state=777)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=777)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [134]:
lgbm_archive_4040 = lgbm_rfe_777(x_train, y_train)

78 0.8644707505645941
76 0.8645658097893718
74 0.8649417258146288
72 0.8651966573719869
70 0.8648923238235701
68 0.8642606120661842
66 0.8644999884776696
64 0.8641605118219109
62 0.8645796365857031
60 0.8644622528460155
58 0.8641812520164077
56 0.8643114543485274
54 0.8644093941558741
52 0.8634052230723142
50 0.8623576992210905
48 0.8631138521454579
46 0.8643370915333917
44 0.8648796492602665
42 0.8637647197769277
40 0.8634772376365396
39 0.8633317682168042
38 0.8633500599161175
37 0.8635882840945753
36 0.8634704682675024
35 0.8636297644835691
34 0.8633182294787299
33 0.8623919781536618
32 0.8627915149559847
31 0.8627726471401576
30 0.863310307876665


In [135]:
model = RandomForestClassifier(n_estimators=1000, n_jobs=64, random_state=777)

x_train_1 = x_train[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]]

model.fit(x_train_1, y_train)

pred_y1 = model.predict_proba(test[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]])
pred_y1 = pred_y1[:,1]

In [136]:
def lgbm_rfe_1234(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        # model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        model = RandomForestClassifier(n_estimators=1000, n_jobs=64, random_state=1234)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [137]:
lgbm_archive_1234 = lgbm_rfe_1234(x_train, y_train)

78 0.8755096719067063
76 0.8758318196271769
74 0.874746206246992
72 0.8742775754711176
70 0.8746363798933822
68 0.8743368012010747
66 0.8736207736260349
64 0.8735299225062949
62 0.873906695900027
60 0.873875645517331
58 0.8740655416078003
56 0.8736571428242853
54 0.8739525527152122
52 0.8738217386029282
50 0.8731587554316607
48 0.8722657694256082
46 0.8723554705311742
44 0.872867658093886
42 0.8727835633074177
40 0.8721224489091828
39 0.871635849161841
38 0.871178574775934
37 0.8713881648591317
36 0.8709316092320838
35 0.8708843148991812
34 0.8708920774948551
33 0.8701595184660651
32 0.8697893576538331
31 0.869975084942922
30 0.87132937038449


In [138]:
model2 = RandomForestClassifier(n_estimators=1000, n_jobs=64, random_state=1234)

x_train_2 = x_train[lgbm_archive_1234.iloc[lgbm_archive_1234[lgbm_archive_1234['score']==lgbm_archive_1234['score'].max()].index[0],2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[lgbm_archive_1234.iloc[lgbm_archive_1234[lgbm_archive_1234['score']==lgbm_archive_1234['score'].max()].index[0],2]])
pred_y2 = pred_y2[:,1]

In [139]:
def lgbm_rfe_99087(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        # model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        model = RandomForestClassifier(n_estimators=1000, n_jobs=64, random_state=99087)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [140]:
lgbm_archive_99087 = lgbm_rfe_99087(x_train, y_train)

78 0.8791360699741958
76 0.8786869627370792
74 0.8794423974538476
72 0.8792378906901253
70 0.8787941196661665
68 0.8791636163718481
66 0.878712922692982
64 0.8786840782975345
62 0.87874695907961
60 0.8784573613493176
58 0.8786800400821719
56 0.8788034940946868
54 0.8779670066267113
52 0.8780737308898668
50 0.8782557390251402
48 0.878619899517664
46 0.8780360889538079
44 0.8776192874395926
42 0.8782211257506033
40 0.8777986995792757
39 0.8767998181649312
38 0.877167728428863
37 0.8768393349866941
36 0.8761451946102516
35 0.8762713888403342
34 0.8768303932241055
33 0.8770961943281534
32 0.8757491610607584
31 0.8760924093665828
30 0.8769472130256675


In [141]:
model3 = RandomForestClassifier(n_estimators=1000, n_jobs=64, random_state=99087)

x_train_3 = x_train[lgbm_archive_99087.iloc[lgbm_archive_99087[lgbm_archive_99087['score']==lgbm_archive_99087['score'].max()].index[0],2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[lgbm_archive_99087.iloc[lgbm_archive_99087[lgbm_archive_99087['score']==lgbm_archive_99087['score'].max()].index[0],2]])
pred_y3 = pred_y3[:,1]

In [142]:
def lgbm_rfe_42(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        # model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        model = RandomForestClassifier(n_estimators=1000, n_jobs=64, random_state=5277)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=5277)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive



In [143]:
lgbm_archive_42 = lgbm_rfe_42(x_train, y_train)

78 0.8816561288429459
76 0.8811573487890648
74 0.8811377095049514
72 0.8816409661603583
70 0.8804129332772651
68 0.8817602459300473
66 0.8817443612149554
64 0.8809338075257737
62 0.8818340376519733
60 0.8816743240620508
58 0.8811239908873721
56 0.8811524389680365
54 0.8813608175487402
52 0.8825720992777364
50 0.8805004436167705
48 0.8805912753057952
46 0.8815638530889128
44 0.8808132280975771
42 0.8809551796878972
40 0.8805108408848306
39 0.8800426750091265
38 0.8792820859692333
37 0.8792601361811065
36 0.8790852599085963
35 0.8790998449651807
34 0.8791134191762591
33 0.8791061988512174
32 0.8789656913259057
31 0.879255370766579
30 0.8785435911239677


In [144]:
model4 = ExtraTreesClassifier(n_estimators=1000, n_jobs=-1, random_state=5277)

x_train_4 = x_train[lgbm_archive_42.iloc[lgbm_archive_42[lgbm_archive_42['score']==lgbm_archive_42['score'].max()].index[0],2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[lgbm_archive_42.iloc[lgbm_archive_42[lgbm_archive_42['score']==lgbm_archive_42['score'].max()].index[0],2]])
pred_y4 = pred_y4[:,1]

In [151]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)
# pred_all = pred_y4

index = pd.read_csv('./data/test.csv').index.to_list()

submission = pd.DataFrame({
    "index" : index,
    "nerdiness" : pred_all
})
submission.to_csv('./data/model2_random_forest_not_remove.csv', index=False)

# model3

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv').drop('index',axis=1)
# train.drop('introelapse',axis=1,inplace=True)
# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['introelapse']>3000].index.to_list())
train = train.drop(train[train['testelapse']>10000].index.to_list())
train = train.drop(train[train['surveyelapse']>10000].index.to_list()) # 나에 대한 추가 질문

test = pd.read_csv('./data/test.csv').drop('index',axis=1)
# test.drop('introelapse',axis=1,inplace=True)
test.loc[test['familysize']>100,'familysize']= train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()
test.loc[test['introelapse']>3000,'introelapse']= train['introelapse'].mean()
test.loc[test['testelapse']>10000,'testelapse']= train['testelapse'].mean()
test.loc[test['surveyelapse']>10000,'surveyelapse']= train['surveyelapse'].mean()

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] - train['Q3'] - train['Q6'] - train['Q7'] - train['Q10'] + train['Q12'] + train['Q15'] - train['Q16']
train['V'] = -train['Q4'] + train['Q5'] + train['Q8'] - train['Q11'] + train['Q13'] - train['Q14'] - train['Q17'] + train['Q18'] + train['Q20']
train['M'] = -train['Q9'] + train['Q19']
train['introelapse'] = np.log1p(train['introelapse'])
train['testelapse'] = np.log1p(train['testelapse'])
train['surveyelapse'] = np.log1p(train['surveyelapse'])
train.drop('hand',axis=1,inplace=True)
# train.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# train.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# train.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# train.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
train_fill_na = train.fillna(train.mean())


value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] - test['Q3'] - test['Q6'] - test['Q7'] - test['Q10'] + test['Q12'] + test['Q15'] - test['Q16']
test['V'] = -test['Q4'] + test['Q5'] + test['Q8'] - test['Q11'] + test['Q13'] - test['Q14'] - test['Q17'] + test['Q18'] + test['Q20']
test['M'] = -test['Q9'] + test['Q19']

test['introelapse'] = np.log1p(test['introelapse'])
test['testelapse'] = np.log1p(test['testelapse'])
test['surveyelapse'] = np.log1p(test['surveyelapse'])
test.drop('hand',axis=1,inplace=True)
# test.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# test.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# test.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# test.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
test_fill_na = test.fillna(test.mean())

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

y_train = train_fill_na['nerdiness']
train_x = train_fill_na.drop('nerdiness',axis=1)
test_x = test_fill_na.copy()

ss.fit(train_x)
train_x = ss.transform(train_x)
test_x = ss.transform(test_x)
# train_fill_na

In [ ]:
len(train_x[0])

In [ ]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Used',DEVICE)



train_y = y_train.copy()

# train_y = 2 - train_y.to_numpy()
# train_x = train_x.to_numpy()
# test_x = test_x.to_numpy()

train_y_t = torch.tensor(train_y, dtype=torch.float32)
train_x_t = torch.tensor(train_x, dtype=torch.float32)
test_x_t = torch.tensor(test_x, dtype=torch.float32)

train_x_t[:, :26] = (train_x_t[:, :26] - 3.) / 2.
test_x_t[:, :26] = (test_x_t[:, :26] - 3.) / 2

train_x_t[:, 30:40] = (train_x_t[:, 30:40] - 3.) / 2.
test_x_t[:, 30:40] = (test_x_t[:, 30:40] - 3.) / 2

test_len = len(test_x_t)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 48
BATCH_SIZE = 1024
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4, # in colab use 2
    'pin_memory': True
}
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(train_x, train_y)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x_t[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x_t[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x_t, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
            nn.Dropout(0.05),
            nn.Linear(77, 180, bias=False),
            nn.LeakyReLU(0.5, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(180, 32, bias=False),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
            # nn.Linear(128, 32, bias=False),
            # nn.ReLU(inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid()
        ).to(DEVICE)
        # criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        criterion = torch.nn.BCELoss()

        optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=4e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                loss.backward()
                optimizer.step()
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    # running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                    running_acc += (((pred) >
                                    0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'.format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = ((model(xx).detach().to('cpu'))).numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))

df = pd.read_csv('./data/sample_submission.csv')
df.iloc[:, 1:] = prediction

In [ ]:
for i in df['nerdiness']:
    print(i)

In [ ]:
df.to_csv('./data/model3.csv', index=False)

# final Ensemble

In [ ]:
model1 = pd.read_csv('./data/model1.csv', index_col = 'index')
model2 = pd.read_csv('./data/model2.csv', index_col='index')

pred_y = (model1)*(0.7) + (model2)*(0.3)

test = pd.read_csv('./data/test.csv')
index = test['index']

submission = pd.DataFrame({
    'index': index,
    'voted': pred_y['voted']
    })

submission.to_csv('./data/combined_model1_model2.csv', index=False)

In [ ]:
combined_12 = pd.read_csv('./data/combined_model1_model2.csv', index_col = 'index')
model3 = pd.read_csv('./data/model3.csv', index_col='index')
model3['voted'] = model3['voted']-1

In [ ]:
pred_y = (model3)*(0.8) + (combined_12)*(0.2)

test = pd.read_csv('./data/test_x.csv')
index = test['index']

submission = pd.DataFrame({
    'index': index,
    'voted': pred_y['voted']
    })

submission.to_csv('./data/submission_final_fix_lgbm.csv', index=False)